<a href="https://colab.research.google.com/github/AdamAnderB/Bramlett_DSPN_S24/blob/main/book/exercises/cross-validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 12: Cross validation
-----

In this exercise, we'll practice implementing cross validation techniques, including leave-one-out and k-fold cross validation. We'll use the `PimaIndiansDiabetes2` practice dataset, which has medical data on a group of Pima Native American women, including whether or not they have diabetes. This dataset is part of the `mlbench` package. We'll be using each person's medical history to predict whether or not they have been diagnosed with diabetes.

# 1: Data (1 pts)
---

Load the `tidyverse`, `boot`, and `mlbench` packages (you may need to install `boot` and `mlbench`).

Load the `PimaIndiansDiabetes2` dataset using the `data()` function. Drop the `insulin` column (it just has a lot of missing data) and then drop `NA`s from the rest of the dataset. Save your updated dataset to a new variable name. Finally, print the dimensions of your new dataset, and look at the first few lines of data.

In [1]:
# INSERT CODE HERE
library(tidyverse)
install.packages("boot")
install.packages("mlbench")
library(boot)
library(mlbench)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [19]:
data(PimaIndiansDiabetes2, package="mlbench")

pid<-PimaIndiansDiabetes2%>%
  select(!c(insulin))%>%
  na.omit()


head(pid)
dim(pid)

,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,33.6,0.627,50,pos
2,1,85,66,29,26.6,0.351,31,neg
4,1,89,66,23,28.1,0.167,21,neg
5,0,137,40,35,43.1,2.288,33,pos
7,3,78,50,32,31.0,0.248,26,pos
9,2,197,70,45,30.5,0.158,53,pos


[1] 532   8

(Note that in medical contexts, `pedigree` refers to a system of measuring family history of a condition. So here, higher numbers mean greater family history of diabetes. You can read more about this dataset [here](https://rdrr.io/cran/mlbench/man/PimaIndiansDiabetes.html).)

# 2. Leave-one-out Cross Validation (4 pts)

In the tutorial, we learned how to fit leave-one-out cross validation using the `cv.glm` function from the `boot` package. But we can also do this manually using `predict()` like we have in the past.

Let's predict `diabetes`, a dichotomous outcome, using all the other variables in our modified dataset.

First, fit a logistic regression model using all of the observations except the very first one. Then use your fitted model to predict whether your holdout case is positive or negative for diabetes. Remember that logistic regression coefficients are in **log-odds**, meaning that if an output is positive, the probability of the outcome is greater than 50%; if the output is negative, the probability of the outcome is less than 50%.

Compare your result to the actual response in row one above. Did your model correctly classify this observation?

In [18]:
# INSERT CODE HERE
model <- glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
             family = binomial(link = 'logit'),
             data = pid[-1,])
prediction_log_odds<-predict(model, newdata = pid[1, ],type = "link")
prediction_probability<-exp(prediction_log_odds)/(1+exp(prediction_log_odds))
pid[1,]
prediction_log_odds
prediction_probability

,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,33.6,0.627,50,pos


1 
0.9920213

1 
0.729487

The model was very good. It did a great job predicting what would have happened. it said it was .99 log odds and that converted is ~.73.

So we just calculated a single iteration of LOOCV. We used 531 rows of our data to fit a model to predict the outcome of the last row.

Below, use a `for` loop to iterate through the rest of your dataset doing the same thing. You will need to:
* Create a data frame `results` with two columns: one named `actual` which holds the true classification for each observation, and one named `predicted`, which should be filled with `NA`s. This is where you'll store the output of your loop.
* Create a loop that runs through each row of your data, pulls that observation out, trains your model on the remaining data, and then tests the fitted model on your test observation.
* Store your model *predictions* ("pos" or "neg" -- not the log-odds) in the `predicted` column of your `results` dataframe

After you run your loop, print the first few lines of `results`.

In [20]:
# Initialize `results` data frame
# INSERT CODE HERE
results <- data.frame(actual = pid$diabetes, predicted = rep(NA, nrow(pid)))


#for loop
for (i in 1:nrow(pid)) {
  # separate individual observation `i` from the rest of your data
  train_data <- pid[-i, ]
  test_data <- pid[i, ]

  # train your model
  model<-glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
               family = binomial(link = 'logit'),
               data = train_data)

  # test model on hold out observation
  prediction_log_odds<-predict(model, newdata=test_data, type="link")
  prediction_probability<-exp(prediction_log_odds)/(1+exp(prediction_log_odds))

  # classify model prediction as "pos" or "neg" and add to `results`
  results$predicted[i]<-ifelse(prediction_probability > 0.5, "pos", "neg")
}

head(results)


,actual,predicted
,<fct>,<chr>
1,pos,pos
2,neg,neg
3,neg,neg
4,pos,pos
5,pos,neg
6,pos,pos


Now, calculate the overall error of your model. What proportion of cases were incorrectly classified?

In [21]:
incorrect_classifications<-sum(results$actual!=results$predicted)
total_observations<-nrow(results)
overall_error_rate<-incorrect_classifications / total_observations
overall_error_rate

[1] 0.2218045

22% were classified incorrectly

# 3. Compare to `cv.glm` (3 pts)

Now, let's compare this result to the `cv.glm` function. Using the tutorial as a guide, use `cv.glm` to run LOOCV on the data, using the same model (i.e., still using all of the variables to predict diabetes diagnosis).

Note that, because this is a `classification` problem and not a regression problem like in the tutorial, we need to adjust the `cost` argument of `cv.glm`. We can read more about this in the docs:

In [29]:
?cv.glm

Here, we see `cost` is defined as:
> "A function of two vector arguments specifying the cost function for the cross-validation. The first argument to cost should correspond to the **observed responses** and the second argument should correspond to the **predicted or fitted responses** from the generalized linear model."

In the example code (scroll to bottom of the docs), we see that the appropriate cost function for a binary classification is

``
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
``

Where `r` is the vector of observed responses (technically "pos" and "neg", but R treats these as 1 and 0 under the hood), and `pi` is the vector of *probabilities* (not log-odds) fit by the model. Thus, this boils down to our error: what proportion of observations were incorrectly classified. You will need to include this code below.

In [30]:
# INSERT CODE HERE
model <- glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
             family = binomial(link = 'logit'),
             data = pid)

cost<-function(r, pi = 0) {mean(abs(r - pi) > 0.5)}

#cv.glm
cv_result <- cv.glm(pid, model, cost, K = nrow(pid))
overall_error <- cv_result$delta
overall_error

[1] 0.2218045 0.2221154

How do your results compare to your manual LOOCV above?

> * *Write response here*
>They are basically exactly the same. Meaning I think I did it right! Yay! both are ~.2218
> *


# 4. Adjusting K and Reflection (2 pts)

Recall that LOOCV has some drawbacks. In particular, it has quite high *variance* which can lead to poor performance on new test data. We can reduce this variance by increasing K.

Below, re-run your cross validation using `cv.glm` with `k` set to 3, 5, 10, and 15.

In [35]:
set.seed(1089)
#INSERT CODE BELOW
k_values <- c(3, 5, 10, 15)
overall_errors <- numeric(length(k_values))
#I decided to do a for loop because I am short on time and my copy and paste
#method was having issues that I couldn't figure out.
# K = 3
# K = 5
# K = 10
# K = 15
for (i in seq_along(k_values)) {
  cv_result <- cv.glm(pid, model, cost, K = k_values[i])
  overall_errors[i] <- cv_result$delta[1]
}
overall_errors

[1] 0.2293233 0.2218045 0.2142857 0.2199248

#### Reflection

How do your errors compare to your LOOCV error above? How do they change as k increases?
> * *Write response here*
> They are all pretty close. However, the lower K seems to produce lower error, but it is only slight different. This makes since because more data per fold. However, it could also be an effect of the random seed. We would need to look across random seeds to know better.
> *  

If you change the random seed above, you'll get slightly different errors. If you were to do the same with your LOOCV above , would you expect to get different results each time? Why or why not?
> * *Write response here*
The results of the Leave one out cross validation are inherently stable. So I wouldnt' expect a change (aka no randomness). It is going through every possibility. However, the folding splits the data randomly which is why it is different with different seeds.
> *


**DUE:** 5pm March 25, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*
>
>
